<a href="https://colab.research.google.com/github/sojeip/-/blob/main/%EA%B0%95%ED%99%94%ED%95%99%EC%8A%B5_%EC%B0%90%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 실행 전에: Colab이라면 필요 패키지 설치 (한 번만)
# !pip install -q pandas numpy scikit-learn openpyxl

import os
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

TRAIN_PATH = "/content/The Trainset.xlsx"
TEST_PATH  = "/content/test.xlsx"
OUT_PATH   = "/content/test_predicted_rewards.xlsx"
RANDOM_STATE = 42

# -------------------------
# 1) 파일 존재 확인
# -------------------------
if not os.path.exists(TRAIN_PATH):
    raise FileNotFoundError(f"훈련 파일을 찾을 수 없습니다: {TRAIN_PATH}")
if not os.path.exists(TEST_PATH):
    raise FileNotFoundError(f"테스트 파일을 찾을 수 없습니다: {TEST_PATH}")

# -------------------------
# 2) 로드
# -------------------------
df_train = pd.read_excel(TRAIN_PATH)
df_test  = pd.read_excel(TEST_PATH)

# -------------------------
# 3) 필요한 컬럼 확인
# -------------------------
feature_cols = [
    'inning','outs','score_diff',
    'runners','batter_type','pitcher_id',
    'ERA','ERA_RISP','K_rate','GB_percent','IPG',
    'OPS','OPS_RISP','FBP'
]
missing_train = [c for c in feature_cols+['reward'] if c not in df_train.columns]
missing_test  = [c for c in feature_cols if c not in df_test.columns]
if missing_train:
    raise ValueError(f"훈련 파일에 다음 컬럼이 없습니다: {missing_train}")
if missing_test:
    raise ValueError(f"테스트 파일에 다음 컬럼이 없습니다: {missing_test}")

# -------------------------
# 4) 전처리: 숫자/범주 분리
# -------------------------
numeric_cols = ['inning','outs','score_diff','ERA','ERA_RISP','K_rate','GB_percent','IPG','OPS','OPS_RISP','FBP']
categorical_cols = ['runners','batter_type','pitcher_id']

# Build transformers (sparse_output for new sklearn; it's fine in modern Colab)
num_transformer = Pipeline([('scaler', StandardScaler())])
cat_transformer = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, numeric_cols),
        ('cat', cat_transformer, categorical_cols)
    ],
    remainder='drop'
)

# -------------------------
# 5) 모델 파이프라인 (RandomForest 회귀)
# -------------------------
model = Pipeline([
    ('preproc', preprocessor),
    ('rf', RandomForestRegressor(n_estimators=200, random_state=RANDOM_STATE, n_jobs=1))
])

# -------------------------
# 6) 학습 (reward 예측)
# -------------------------
X_train = df_train[feature_cols]
y_train = df_train['reward']

print("학습 시작... (훈련 데이터 행 수:", len(X_train), ")")
model.fit(X_train, y_train)
print("학습 완료.")

# -------------------------
# 7) 테스트에 대해 예측 (각 행은 한 투수)
# -------------------------
X_test = df_test[feature_cols]
pred = model.predict(X_test)

# 안전 체크
if np.any(np.isnan(pred)) or np.any(np.isinf(pred)):
    raise ValueError("예측에 NaN/inf가 포함되어 있습니다.")

# -------------------------
# 8) 결과표: 테스트 파일의 pitcher_id와 predicted_reward만 저장
# -------------------------
results = pd.DataFrame({
    'pitcher_id': df_test['pitcher_id'].astype(str).values,
    'predicted_reward': np.round(pred, 6)
})

results.to_excel(OUT_PATH, index=False)
print("예측 완료. 결과 파일:", OUT_PATH)
print("\n== 예측 결과 (콘솔 출력) ==")
print(results.to_string(index=False))


학습 시작... (훈련 데이터 행 수: 6000 )
학습 완료.
예측 완료. 결과 파일: /content/test_predicted_rewards.xlsx

== 예측 결과 (콘솔 출력) ==
pitcher_id  predicted_reward
       김서현          1.147395
       황준서          1.114090
       김종수          1.121630
       조동욱          1.099555
       윤산흠          1.116990
